In [1]:
import torch

print(torch.cuda.is_available())


print(torch.cuda.device_count())


print(torch.cuda.current_device())


print(torch.cuda.device(0))


print(torch.cuda.get_device_name(0))




True
1
0
NVIDIA GeForce RTX 3060 Laptop GPU


In [2]:
import torch
import torch.nn as nn
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
import torch.optim as optim


In [ ]:
import os
import torch
from torchvision import datasets, transforms
from torchvision.transforms import functional as F
from torch.utils.data import DataLoader, random_split
from torchvision.transforms import v2

transforms = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.RandomHorizontalFlip(p=0.5),  
    transforms.ToTensor(),  
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  
])

# Load the dataset with the custom transform
dataset = datasets.ImageFolder(root='C:/Users/cnaja/IISC/NJN/data/', transform= transforms)


# Split the dataset into train, validation, and test sets
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - (train_size + val_size)
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create DataLoader instances
train_loader = DataLoader(train_dataset, batch_size= 32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size= 32, shuffle=False)



In [ ]:
import matplotlib.pyplot as plt

# Function to show a batch of images
def show_images(images, nmax=15):
    fig, axes = plt.subplots(nrows=1, ncols=min(nmax, len(images)), figsize=(20, 3))
    for i, ax in enumerate(axes):
        image = images[i].permute(1, 2, 0)  WC
        ax.imshow(image)
        ax.axis('off')
    plt.show()

dataiter = iter(train_loader)
images, labels = next(dataiter)

show_images(images)



In [ ]:
from torch.optim import lr_scheduler
from torch import nn, optim
import torch.optim as optim
import torch
from torchmetrics import Accuracy, Precision, Recall, F1Score
from torch.nn.utils import clip_grad_norm_



# Initialize the metrics
task = 'binary'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
accuracy = Accuracy(task=task).to(device)
precision = Precision(task=task, num_classes=2, average='macro').to(device)
recall = Recall(task=task, num_classes=2, average='macro').to(device)
f1_score = F1Score(task=task, num_classes=2, average='macro').to(device)





model = models.resnet50(pretrained=True)
model.fc = nn.Linear(num_ftrs, 2)
num_ftrs = model.fc.in_features
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr= 1e-4)
scheduler = StepLR(optimizer, step_size=7, gamma=0.1)



# Move model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)





early_stopping_patience = 20
min_val_loss = float('inf')
patience_counter = 0





def reset_metrics():
    accuracy.reset()
    precision.reset()
    recall.reset()
    f1_score.reset()

def update_metrics(preds, labels):
    preds = torch.sigmoid(preds) >= 0.5
    accuracy(preds, labels.int())
    precision(preds, labels.int())
    recall(preds, labels.int())
    f1_score(preds, labels.int())



def print_metrics(epoch, phase="Validation"):
    print(f"{phase} Metrics Epoch {epoch+1}: Accuracy: {accuracy.compute():.4f}, Precision: {precision.compute():.4f}, Recall: {recall.compute():.4f}, F1 Score: {f1_score.compute():.4f}")


train_acc, train_prec, train_recall, train_f1 = [], [], [], []
val_acc, val_prec, val_recall, val_f1 = [], [], [], []

def train(epoch):
    model.train()
    running_loss = 0.0
    reset_metrics()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device).float().view(-1, 1)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        clip_grad_norm_(model.parameters(), max_norm=1)
        optimizer.step()
        
        running_loss += loss.item()
        update_metrics(outputs, labels)
        
    epoch_loss = running_loss / len(train_loader)
    train_acc.append(accuracy.compute().item())
    train_prec.append(precision.compute().item())
    train_recall.append(recall.compute().item())
    train_f1.append(f1_score.compute().item())
    reset_metrics()

    print(f"Epoch {epoch+1}, Loss: {epoch_loss}")
    print(f"Training Metrics Epoch {epoch+1}: Accuracy: {train_acc[-1]:.4f}, Precision: {train_prec[-1]:.4f}, Recall: {train_recall[-1]:.4f}, F1 Score: {train_f1[-1]:.4f}")


def validate(epoch):
    global min_val_loss, patience_counter
    model.eval()
    val_loss = 0.0
    reset_metrics()
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device).float().view(-1, 1)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            update_metrics(outputs, labels)
    
    epoch_val_loss = val_loss / len(val_loader)
    val_acc.append(accuracy.compute().item())
    val_prec.append(precision.compute().item())
    val_recall.append(recall.compute().item())
    val_f1.append(f1_score.compute().item())
    reset_metrics()

    print(f"Epoch {epoch+1}, Validation Loss: {epoch_val_loss}")
    print(f"Validation Metrics Epoch {epoch+1}: Accuracy: {val_acc[-1]:.4f}, Precision: {val_prec[-1]:.4f}, Recall: {val_recall[-1]:.4f}, F1 Score: {val_f1[-1]:.4f}")

    if epoch_val_loss < min_val_loss:
        min_val_loss = epoch_val_loss
        patience_counter = 0
        torch.save(model.state_dict(), best_model_path)
        print(f"Validation loss decreased to {min_val_loss:.6f}. Saving the model!")
    else:
        patience_counter += 1
        if patience_counter >= early_stopping_patience:
            print("Early stopping triggered.")
            return True
    return False


best_model_path = 'Gnet_NJN_resnet1.pth'
num_epochs = 500  

for epoch in range(num_epochs):
    train(epoch)
    stop_training = validate(epoch)
    if stop_training:
        print("Stopping training...")
        break
    scheduler.step() 


model.load_state_dict(torch.load(best_model_path))
model.eval()







In [ ]:
def test():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device).float().view(-1, 1)
            outputs = model(inputs)
            predicted = torch.sigmoid(outputs).round()  
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Accuracy on the test set: {100 * correct / total}%")

# Run the test
test()

In [ ]:
import matplotlib.pyplot as plt

# Function to plot metrics
def plot_metrics(train_metrics, val_metrics, metric_name):
    plt.figure(figsize=(10, 6))
    plt.plot(train_metrics, label=f'Train {metric_name}')
    plt.plot(val_metrics, label=f'Validation {metric_name}')
    plt.xlabel('Epochs')
    plt.ylabel(metric_name)
    plt.title(f'Train vs Validation {metric_name}')
    plt.legend()
    plt.show()

# Plot accuracy, precision, recall, and F1 score
plot_metrics(train_acc, val_acc, 'Accuracy')
plot_metrics(train_prec, val_prec, 'Precision')
plot_metrics(train_recall, val_recall, 'Recall')
plot_metrics(train_f1, val_f1, 'F1 Score')
